#### Task 3: 3D Visualization 

In [1]:
import open3d as o3d
import numpy as np

print(o3d.__version__)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
0.19.0


In [2]:
# Step 1 Loading  the original point cloud
pcd = o3d.io.read_point_cloud("fragment.pcd")

num_points = np.asarray(pcd.points).shape[0]
print(f"Number of points: {num_points}")


Number of points: 113662


#### About the Data

In [3]:
# Compute bounding box
bbox = pcd.get_axis_aligned_bounding_box()
min_bound = bbox.min_bound
max_bound = bbox.max_bound
print(f"Bounding box min: {min_bound}")
print(f"Bounding box max: {max_bound}")

Bounding box min: [1.15198874 0.81527287 0.72640908]
Bounding box max: [3.51953125 2.76171875 1.53042805]


In [4]:
# Compute density (approximate)
vol = np.prod(max_bound - min_bound)
density = num_points / vol
print(f"Approximate point density: {density:.2f} points per unit volume")

Approximate point density: 30676.72 points per unit volume


In [5]:
# Check if color is available
if pcd.has_colors():
    print("Point cloud has color information.")
else:
    print("No color information found.")

Point cloud has color information.


In [6]:
# Check if normals are available
if pcd.has_normals():
    print("Point cloud has normals.")
else:
    print("No normals found.")

Point cloud has normals.


In [7]:
# Show some sample points and normals
points = np.asarray(pcd.points)
normals = np.asarray(pcd.normals)
print("Sample points (first 5):")
print(points[:5])
print("Sample normals (first 5):")
print(normals[:5])

Sample points (first 5):
[[1.16796875 1.01803279 0.96484375]
 [1.16845131 1.01953125 0.96484375]
 [1.16796875 1.02158833 0.96484375]
 [1.16796875 1.01953125 0.96634495]
 [1.16796875 1.03289878 0.95703125]]
Sample normals (first 5):
[[ 0.96546179 -0.08612899  0.24589692]
 [ 0.96990991 -0.07807195  0.23060687]
 [ 0.98010278 -0.0494331   0.1922366 ]
 [ 0.9759478  -0.06353842  0.20853986]
 [ 0.97493398 -0.2121565   0.06703294]]


In [8]:
# Visualize the original point cloud
print("Visualizing original point cloud...")
o3d.visualization.draw_geometries([pcd])

Visualizing original point cloud...


### Mathematical Explanation of Voxel Downsampling

Voxel downsampling divides the point cloud's 3D space into small cubes (voxels) of side length `s` (voxel size). All points inside a voxel are replaced by a single representative point, reducing the total number of vertices.

Let:

- \( s \) = voxel size (e.g., 0.03)
- \( V \) = volume of the point cloud bounding box
- \( N_o \) = number of points in the original point cloud
- \( N_d \) = number of points in the downsampled point cloud


Each voxel is a cube with volume:

\[
v = s^3
\]

The maximum number of possible voxels is:

\[
N_{\text{max\_voxels}} = \frac{V}{s^3}
\]

Only the **occupied** voxels contribute to the final downsampled point cloud, so:

\[
N_d \leq \frac{V}{s^3}
\]

This means the number of vertices in the downsampled point cloud is **inversely proportional to the cube of the voxel size**:

\[
N_d \propto \frac{1}{s^3}
\]

So, decreasing the voxel size increases the number of downsampled points significantly.  
For example, halving the voxel size increases the point count by approximately:

\[
2^3 = 8 \text{ times}
\]



In [9]:
# Step 2: Downsample the point cloud using voxel downsampling
voxel_size = 0.03 #voxel size can be adjusted as needed
downsampled_pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
print(f"Downsampled point cloud has {len(downsampled_pcd.points)} points")

Downsampled point cloud has 7110 points


In [10]:
# Estimate normals for downsampled point cloud
downsampled_pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
)


In [11]:
# Visualize the downsampled point cloud with normals
print("Visualizing downsampled point cloud...")
o3d.visualization.draw_geometries([downsampled_pcd], point_show_normal=True)

Visualizing downsampled point cloud...


In [12]:
# Add this cell to your notebook for a professional touch:
def compare_point_clouds(original_pcd, downsampled_pcd):
    original_count = len(original_pcd.points)
    downsampled_count = len(downsampled_pcd.points)
    
    reduction_percentage = (original_count - downsampled_count) / original_count * 100
    compression_ratio = original_count / downsampled_count
    
    print(f"📊 POINT CLOUD COMPARISON")
    print(f"Original vertices:     {original_count:,}")
    print(f"Downsampled vertices:  {downsampled_count:,}")
    print(f"Reduction:             {reduction_percentage:.2f}%")
    print(f"Compression ratio:     {compression_ratio:.2f}x")

# Use it:
compare_point_clouds(pcd, downsampled_pcd)

📊 POINT CLOUD COMPARISON
Original vertices:     113,662
Downsampled vertices:  7,110
Reduction:             93.74%
Compression ratio:     15.99x


In [14]:
#  Normal Quality Assessment
def analyze_normals(pcd_with_normals):
    """
    Add this to analyze the quality of estimated normals
    """
    normals = np.asarray(pcd_with_normals.normals)
    
    # Check normal magnitudes (should be close to 1.0)
    magnitudes = np.linalg.norm(normals, axis=1)
    
    print(f"🔍 NORMAL VECTOR ANALYSIS")
    print(f"{'='*40}")
    print(f"Number of normals:     {len(normals):,}")
    print(f"Magnitude mean:        {magnitudes.mean():.6f}")
    print(f"Magnitude std:         {magnitudes.std():.6f}")
    print(f"Magnitude range:       [{magnitudes.min():.6f}, {magnitudes.max():.6f}]")
    
    # Check if normals are unit vectors (magnitude ≈ 1.0)
    unit_normals = np.sum(np.abs(magnitudes - 1.0) < 0.001)
    print(f"Unit normals:          {unit_normals}/{len(normals)} ({unit_normals/len(normals)*100:.1f}%)")

analyze_normals(downsampled_pcd)

🔍 NORMAL VECTOR ANALYSIS
Number of normals:     7,110
Magnitude mean:        1.000000
Magnitude std:         0.000000
Magnitude range:       [1.000000, 1.000000]
Unit normals:          7110/7110 (100.0%)


In [16]:
#  Side-by-Side Visualization
def visualize_comparison(pcd, downsampled_pcd):
    """
    Show both point clouds side by side
    """
    # Color the point clouds differently
    original_colored = pcd.paint_uniform_color([0.7, 0.7, 0.7])  # Gray
    downsampled_colored = downsampled_pcd.paint_uniform_color([1.0, 0.0, 0.0])  # Red
    
    print("Visualizing original (gray) vs downsampled (red) point clouds...")
    o3d.visualization.draw_geometries([original_colored, downsampled_colored])

visualize_comparison(pcd, downsampled_pcd)


Visualizing original (gray) vs downsampled (red) point clouds...
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


Summary and Analysis of Normal Estimation Visualization

The visualization shows the downsampled point cloud with estimated surface normals represented as small arrows at each point. The normals are generally consistent and smoothly oriented across flat surfaces, indicating accurate normal estimation. At edges and corners, the normals appropriately diverge, reflecting changes in surface orientation. The downsampling retains sufficient point density to capture the main structural features. Overall, the chosen parameters for normal estimation (radius=0.1, max_nn=30) produce reliable and meaningful normal directions that effectively represent the underlying geometry of the scene.